<a href="https://colab.research.google.com/github/kgautam2103/Meetup_automation/blob/main/Announce_event_meetup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
client_key=""

client_redirect_url="https://us.sahajayoga.org/"

authorize_url = "https://secure.meetup.com/oauth2/authorize?client_id="+client_key+"&response_type=code&redirect_uri="+client_redirect_url

print(authorize_url)

https://secure.meetup.com/oauth2/authorize?client_id=emgo5alo5672bdgr1k4bp6sn57&response_type=code&redirect_uri=https://us.sahajayoga.org/


In [ ]:
import requests
import json
import pandas as pd
import time
from datetime import datetime
from dateutil import tz

def get_access_refresh_token_from_code(access_code,client_secret, grant_type_code):
  access_url = "https://secure.meetup.com/oauth2/access?client_id="+client_key+"&client_secret="+client_secret+"&grant_type="+grant_type_code+"&redirect_uri="+client_redirect_url+"&code="+access_code


  response = requests.post(access_url)

  print(response.json())

  access_token = response.json()["access_token"]
  refresh_token = response.json()["refresh_token"]

  return access_token,refresh_token


def get_access_refresh_token_from_refresh(refresh_token,client_secret, grant_type_refresh):
  access_url = "https://secure.meetup.com/oauth2/access?client_id="+client_key+"&client_secret="+client_secret+"&grant_type="+grant_type_refresh+"&refresh_token="+refresh_token


  response = requests.post(access_url)

  print(response.json())

  access_token = response.json()["access_token"]
  refresh_token = response.json()["refresh_token"]

  return access_token,refresh_token


access_code = input("please enter the access code recieved ")

client_secret=""

grant_type_code = "authorization_code"

grant_type_refresh = "refresh_token"

meetup_api_url = "https://api.meetup.com/gql"

#sheet_name = input('enter the start time in UTC in format 2022-12-31T13:45 : ')

access_token,refresh_token = get_access_refresh_token_from_code(access_code,client_secret, grant_type_code)

please enter the access code recieved 29e31adb712599ab52cde195e8daed97
{'access_token': 'd0918f8ce8add9a79e44570893305387', 'refresh_token': 'af76755bb0718cf7d83b6d600cc49d3d', 'token_type': 'bearer', 'expires_in': 3600}


In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1p-0YsX45ZSIcU4elqs8kIa4wBkDZNoZ-_MOBHY1JHog/edit#gid=0')

current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_sheet = current_ts+'_event_announce_output_meetup'

counter=1

# create and Open our new sheet and add some data.
sh = gc.create(output_sheet)
output_sheet = gc.open(output_sheet).sheet1

output_sheet.insert_row(['event_id','event_url','is_announced','errors'],1)

sheet_name = input('enter the name of input sheet : ')
input_sheet = wb.worksheet(sheet_name)
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['event_id','event_url'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])

for index,row in inputdata_df.iterrows():
  counter = counter+1
  is_announced="False"
  announce_event_error=""
  access_token,refresh_token = get_access_refresh_token_from_refresh(refresh_token,client_secret, grant_type_refresh)
  authorization = 'Bearer ' + access_token
  auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

  print("access_token "+access_token)
  print("refresh_token "+refresh_token)
  cell_value_list = []
  event_id = row['event_id']
  event_url = row['event_url']

  print(event_id)

  announce_event_query = """mutation($announce_event_input: AnnounceEventInput!) {
  announceEvent(input: $announce_event_input) {
      event {
        id
      }
      errors {
        message
        code
        field
      }
    }
  }"""

  announce_event_variables = {
    "announce_event_input": {
      "eventId": event_id
    }
  }

  announce_event_mutation_response = requests.post(meetup_api_url, json={'query': announce_event_query, 'variables': announce_event_variables}, headers=auth_headers)

  print(announce_event_mutation_response.text)

  try:
    announce_event_error=announce_event_mutation_response.json()['data']['announceEvent']['errors'][0]['message']
  except:
    announce_event_error=""
    is_announced="True"


  cell_value_list = [event_id,event_url,is_announced,announce_event_error]
  output_sheet.insert_row(cell_value_list,counter)

enter the name of input sheet : inputdata
{'access_token': '605f58e85cdeea9efc024a3dfa347273', 'refresh_token': 'af76755bb0718cf7d83b6d600cc49d3d', 'token_type': 'bearer', 'expires_in': 3600}
access_token 605f58e85cdeea9efc024a3dfa347273
refresh_token af76755bb0718cf7d83b6d600cc49d3d
290223309
{"data":{"announceEvent":{"event":{"id":"290223309"},"errors":[]}}}

{'access_token': '8f8295d55a89aa6c5ba0d89c32504603', 'refresh_token': 'af76755bb0718cf7d83b6d600cc49d3d', 'token_type': 'bearer', 'expires_in': 3600}
access_token 8f8295d55a89aa6c5ba0d89c32504603
refresh_token af76755bb0718cf7d83b6d600cc49d3d
290223311
{"data":{"announceEvent":{"event":{"id":"290223311"},"errors":[]}}}

{'access_token': '803e40044d06a97465c2e5bbe67b035e', 'refresh_token': 'af76755bb0718cf7d83b6d600cc49d3d', 'token_type': 'bearer', 'expires_in': 3600}
access_token 803e40044d06a97465c2e5bbe67b035e
refresh_token af76755bb0718cf7d83b6d600cc49d3d
290223312
{"data":{"announceEvent":{"event":{"id":"290223312"},"errors"

In [ ]:
print(announce_event_mutation_response.json()['data']['announceEvent']['errors'][0]['message'])

Cannot announce a draft event or an event that's already been announced
